# Fine-tuning - googlenet for distracted driver dataset

In [1]:
caffe_root = '../' 

import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

caffe.set_device(0)
caffe.set_mode_gpu()

import numpy as np
from pylab import *
%matplotlib inline
import tempfile
import os
import time

In [2]:
# Helper function for deprocessing preprocessed images, e.g., for display.
def deprocess_net_image(image):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(1, 2, 0)  # CHW -> HWC
    image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image

Define the num of labels and labels

In [3]:
NUM_DD_LABELS = 10
dd_labels = list(range(0,NUM_DD_LABELS))
print dd_labels

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Define Solver

In [28]:
from caffe.proto import caffe_pb2

def solver(train_net_path, test_net_path=None, base_lr=0.0001, snapshot_prefix = 'ddxx'):
    
    s = caffe_pb2.SolverParameter()

    # Specify locations of the train and (maybe) test networks.
    s.train_net = train_net_path
    if test_net_path is not None:
        s.test_net.append(test_net_path)
        s.test_interval = 1000  # Test after every 1000 training iterations.
        s.test_iter.append(15) # Test on 100 batches each time we test.
        
    # The number of iterations over which to average the gradient.
    # Effectively boosts the training batch size by the given factor, without
    # affecting memory utilization.
    s.iter_size = 1
    
    s.max_iter = 100000     # # of times to update the net (training iterations)
    
    # Solve using the stochastic gradient descent (SGD) algorithm.
    # Other choices include 'Adam' and 'RMSProp'.
    s.type = 'Adam'

    # Set the initial learning rate for SGD.
    s.base_lr = base_lr

    # Set `lr_policy` to define how the learning rate changes during training.
    # Here, we 'step' the learning rate by multiplying it by a factor `gamma`
    # every `stepsize` iterations.
    s.lr_policy = 'step'
    s.gamma = 0.1
    s.stepsize = 20000

    # Set other SGD hyperparameters. Setting a non-zero `momentum` takes a
    # weighted average of the current gradient and previous gradients to make
    # learning more stable. L2 weight decay regularizes learning, to help prevent
    # the model from overfitting.
#    s.momentum = 0.9
#    s.weight_decay = 5e-4

    # Display the current training loss and accuracy every 1000 iterations.
    s.display = 1000

    # Snapshots are files used to store networks we've trained.  Here, we'll
    # snapshot every 10K iterations -- ten times during training.
    s.snapshot = 100
    s.snapshot_prefix = caffe_root + 'models/dd/' + snapshot_prefix
    
    # Train on the GPU.  Using the CPU to train large networks is very slow.
    s.solver_mode = caffe_pb2.SolverParameter.GPU
    
    # Write the solver to a temporary file and return its filename.
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str(s))
        return f.name

In [29]:
train_net_path = '/home/ubuntu/caffe2/caffe/models/googlenet_10class_finetune/googlenet10class_tuneallfc.prototxt'
weights = '/home/ubuntu/caffe2/caffe/models/dd/google_fine_3_iter_900.caffemodel'
#weights = '/home/ubuntu/caffe2/caffe/models/googlenet_10class_finetune/bvlc_googlenet.caffemodel'

### Solver

In [30]:
solver_path = solver(train_net_path, base_lr=0.0001, snapshot_prefix='google_fine_4')
solver = caffe.get_solver(solver_path)
solver.net.copy_from(weights)

## Train

In [32]:
niter = 1000
disp_interval = 50

blobs = ('loss', 'acc')
loss = np.zeros(niter)
acc = np.zeros(niter)

start_time = time.time()
for it in range(niter):
    solver.step(1)  # run a single SGD step in Caffe
    loss[it] = solver.net.blobs['loss1/loss1'].data.copy()
    #acc[it] = solver.net.blobs['acc'].data.copy()
        
    if it % disp_interval == 0 or it + 1 == niter:
        #test_loss = 0
        #test_acc = 0
        #for i in range(0,20):
        #    solver.test_nets[0].forward()
        #    test_loss = test_loss + solver.test_nets[0].blobs['loss'].data
        #    test_acc = test_acc + solver.test_nets[0].blobs['acc'].data
        print it, 'train loss=', loss[it] , ' time=', time.time()- start_time

0 train loss= 1.58110380173  time= 0.326205968857
50 train loss= 1.57639837265  time= 14.8060650826
100 train loss= 1.3762332201  time= 29.6404249668
150 train loss= 0.919660866261  time= 44.1239509583
200 train loss= 1.18341779709  time= 58.9143519402
250 train loss= 0.909773826599  time= 73.3944039345
300 train loss= 0.913184523582  time= 88.1747879982
350 train loss= 0.899425506592  time= 102.654928923
400 train loss= 1.05257618427  time= 117.445826054
450 train loss= 0.729639530182  time= 131.932982922
500 train loss= 0.925218999386  time= 146.730884075
550 train loss= 0.492480874062  time= 161.210880995
600 train loss= 0.537905216217  time= 176.000396967
650 train loss= 0.528548002243  time= 190.482832909
700 train loss= 0.53393638134  time= 205.273307085
750 train loss= 0.56178945303  time= 219.753696918
800 train loss= 0.399997919798  time= 234.547713995
850 train loss= 0.38559371233  time= 249.033856869
900 train loss= 0.384254157543  time= 263.859920979
950 train loss= 0.58039

In [17]:
float(test_loss)

3.221517562866211

## Get output ( deploy)

In [22]:
source = '/home/ubuntu/distracteddriver/data/imgs/output1.txt'

In [12]:
outputnet_path = '/home/ubuntu/caffe2/caffe/models/googlenet_10class_finetune/googlenet10class_deploy.prototxt'
weights = '/home/ubuntu/caffe2/caffe/models/dd/google_fine_4_iter_900.caffemodel'
outputnet = caffe.Net(outputnet_path,weights, caffe.TEST)

In [13]:
import time
max_iter = 79726/32 + 1
start_time = time.time()
output = np.zeros((1,10))
for i in range(0,max_iter):
    if (i%100 == 0):
        print i , time.time() - start_time        
    output = np.append(output,outputnet.forward()['prob'], axis=0)

0 0.000503063201904
100 27.6660430431
200 55.3169541359
300 82.9712870121
400 110.631191969
500 138.291491985
600 165.959401131
700 193.634925127
800 221.320247173
900 249.015668154
1000 276.724761009
1100 304.436163187
1200 332.152372122
1300 359.863602161
1400 387.591262102
1500 415.331500053
1600 443.085996151
1700 470.837129116
1800 498.597136974
1900 526.365695
2000 554.136285067
2100 581.917642117
2200 609.709833145
2300 637.513375044
2400 665.314886093


In [14]:
outfile = '/home/ubuntu/distracteddriver/outfile_google_fine_allfc.csv'

In [15]:
np.savetxt(outfile, output, delimiter = ',')

In [48]:
import matplotlib.pyplot as plt
x = np.arange(0,train_loss.shape[0])
plt.plot(x,train_loss)
plt.plot(x, train_acc)